In [1]:
from random import seed
from random import randint 
from numpy import array,argmax
from math import ceil,log10,sqrt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, RepeatVector
import numpy as np

2023-03-18 21:15:17.670755: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-18 21:15:17.899989: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-18 21:15:17.900015: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-18 21:15:17.958705: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-18 21:15:19.008151: W tensorflow/stream_executor/platform/de

In [2]:
def random_num_pairs(n_examples, n_numbers, largest):
    X,y = [], []
    for i in range(n_examples):
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        out_pattern = np.sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y


In [3]:
X,y = random_num_pairs(10,2,300)
print(X)
print(y)

[[29, 192], [203, 61], [20, 114], [189, 168], [13, 167], [210, 86], [120, 24], [227, 284], [99, 218], [296, 172]]
[221, 264, 134, 357, 180, 296, 144, 511, 317, 468]


In [4]:
def pairs_to_string(X,y, n_numbers, largest):
    max_length = n_numbers* ceil(log10(largest+1))+n_numbers -1
    Xstr= []
    for p in X:
        strp = '+'.join([str(n) for n in p])        
        strp = ''.join([' ' for _ in range(max_length - len(strp))])+ strp
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest+1)))
    ystr= []
    for p in y:
        strp = str(p)
        strp = ''.join([' ' for _ in range(max_length - len(strp))])+ strp
        ystr.append(strp)
    return Xstr, ystr


In [5]:
Xstr,ystr =pairs_to_string(X,y, 2,300)

In [6]:
print(Xstr,ystr)

[' 29+192', ' 203+61', ' 20+114', '189+168', ' 13+167', ' 210+86', ' 120+24', '227+284', ' 99+218', '296+172'] ['221', '264', '134', '357', '180', '296', '144', '511', '317', '468']


In [7]:
def integer_encode(X, y, alphsbets):
    char_to_int = dict((c, i) for i, c in enumerate(alphsbets))
    Xenc = []
    for p in X:
        integer_encoded = [char_to_int[char] for char in p]
        Xenc.append(integer_encoded)
    yenc = []
    for p in y:
        integer_encoded = [char_to_int[char] for char in p]
        yenc.append(integer_encoded)
    return Xenc, yenc


In [8]:
integer_encode(Xstr,ystr,['1','2','3','4','5','6','7','8','9','0',' ','+'])

([[10, 1, 8, 11, 0, 8, 1],
  [10, 1, 9, 2, 11, 5, 0],
  [10, 1, 9, 11, 0, 0, 3],
  [0, 7, 8, 11, 0, 5, 7],
  [10, 0, 2, 11, 0, 5, 6],
  [10, 1, 0, 9, 11, 7, 5],
  [10, 0, 1, 9, 11, 1, 3],
  [1, 1, 6, 11, 1, 7, 3],
  [10, 8, 8, 11, 1, 0, 7],
  [1, 8, 5, 11, 0, 6, 1]],
 [[1, 1, 0],
  [1, 5, 3],
  [0, 2, 3],
  [2, 4, 6],
  [0, 7, 9],
  [1, 8, 5],
  [0, 3, 3],
  [4, 0, 0],
  [2, 0, 6],
  [3, 5, 7]])

In [9]:
def one_hot_encode(X, y, max_int):
    Xenc = []
    for p in X:
        pattern = []
        for index in p:
            Vector = [0 for _ in range(max_int)]
            Vector[index] = 1
            pattern.append(Vector)
        Xenc.append(pattern)
    yenc = []
    for p in y:
        pattern = []
        for index in p:
            Vector = [0 for _ in range(max_int)]
            Vector[index] = 1
            pattern.append(Vector)
        yenc.append(pattern)
    return Xenc, yenc

In [10]:
def integer_decode(seq, alphsbets):
    int_to_char = dict((i,c) for i, c in enumerate(alphsbets))
    strings =[]
    for p in seq:
        string = int_to_char[argmax(p)]
        strings.append(string)
    return ''.join(strings)

In [11]:
def generate_data(n_samples, n_numbers, largest, alphsbets):
    X,y = random_num_pairs(n_samples, n_numbers, largest)
    Xstr,ystr =pairs_to_string(X,y, n_numbers, largest)
    Xenc, yenc = integer_encode(Xstr,ystr,alphsbets)
    X,y = one_hot_encode(Xenc, yenc,len(alphsbets))
    X,y =np.array(X),np.array(y)
    return X,y


In [12]:
seed(1)
n_samples =10000
n_numbers = 3
largest = 20
alphsbets=['1','2','3','4','5','6','7','8','9','0',' ','+']
n_chars = len(alphsbets)
n_in_seq_length = n_numbers* ceil(log10(largest+1))+n_numbers -1
n_out_seq_length =  ceil(log10(n_numbers *(largest+1)))

In [13]:
X,y = generate_data(n_samples, n_numbers, largest, alphsbets)

In [14]:
X.shape

(10000, 8, 12)

In [15]:
n_in_seq_length

8

In [16]:
y.shape

(10000, 2, 12)

In [17]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_in_seq_length,n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))

2023-03-18 21:15:21.991089: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-03-18 21:15:21.991156: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: momeni
2023-03-18 21:15:21.991183: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: momeni
2023-03-18 21:15:21.991380: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.108.3
2023-03-18 21:15:21.991454: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.108.3
2023-03-18 21:15:21.991477: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.108.3
2023-03-18 21:15:21.991985: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical ope

In [18]:
model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics='accuracy')

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               72192     
                                                                 
 repeat_vector (RepeatVector  (None, 2, 128)           0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 2, 64)             49408     
                                                                 
 time_distributed (TimeDistr  (None, 2, 12)            780       
 ibuted)                                                         
                                                                 
Total params: 122,380
Trainable params: 122,380
Non-trainable params: 0
_________________________________________________________________


In [20]:
X_test,y_test = generate_data(int(n_samples/20), n_numbers, largest, alphsbets)

In [21]:
for i in range(50):
    X,y = generate_data(n_samples, n_numbers, largest, alphsbets)
    print('epoch: ',i)
    model.fit(X,y, epochs=5,validation_data=(X_test,y_test), batch_size=50)

epoch:  0
Epoch 1/5
200/200 [==============================] - 9s 21ms/step - loss: 1.9194 - accuracy: 0.2774 - val_loss: 1.6103 - val_accuracy: 0.3770
Epoch 2/5
200/200 [==============================] - 3s 16ms/step - loss: 1.5363 - accuracy: 0.3938 - val_loss: 1.4544 - val_accuracy: 0.4310
Epoch 3/5
200/200 [==============================] - 3s 16ms/step - loss: 1.4241 - accuracy: 0.4394 - val_loss: 1.3702 - val_accuracy: 0.4580
Epoch 4/5
200/200 [==============================] - 3s 15ms/step - loss: 1.3540 - accuracy: 0.4760 - val_loss: 1.3008 - val_accuracy: 0.5160
Epoch 5/5
200/200 [==============================] - 3s 15ms/step - loss: 1.3088 - accuracy: 0.5005 - val_loss: 1.2501 - val_accuracy: 0.5360
epoch:  1
Epoch 1/5
200/200 [==============================] - 3s 15ms/step - loss: 1.2445 - accuracy: 0.5391 - val_loss: 1.1715 - val_accuracy: 0.5950
Epoch 2/5
200/200 [==============================] - 3s 15ms/step - loss: 1.1573 - accuracy: 0.5687 - val_loss: 1.1042 - val_acc

In [22]:
result = model.predict(X_test,batch_size=50)

10/10 [==============================] - 1s 7ms/step


In [23]:
integer_decode(result[0],alphsbets)

'37'

In [24]:
exp = [integer_decode(y1, alphsbets) for y1 in y_test]
pred = [integer_decode(y1, alphsbets) for y1 in result]
list(zip(exp, pred))

[('37', '37'),
 ('38', '38'),
 (' 8', ' 8'),
 ('33', '33'),
 ('40', '40'),
 ('35', '35'),
 ('36', '36'),
 ('38', '38'),
 ('35', '35'),
 ('33', '33'),
 ('25', '25'),
 ('18', '18'),
 ('47', '47'),
 ('33', '33'),
 ('24', '24'),
 ('26', '26'),
 ('32', '32'),
 (' 8', ' 8'),
 ('13', '13'),
 ('50', '50'),
 ('33', '33'),
 ('44', '44'),
 ('40', '40'),
 ('31', '31'),
 ('40', '40'),
 ('39', '39'),
 ('22', '22'),
 ('39', '39'),
 ('13', '13'),
 ('34', '34'),
 ('20', '20'),
 ('26', '26'),
 ('34', '34'),
 ('32', '32'),
 ('38', '38'),
 ('37', '37'),
 ('35', '35'),
 ('42', '42'),
 ('31', '31'),
 ('34', '34'),
 ('36', '36'),
 ('16', '16'),
 ('28', '28'),
 ('34', '34'),
 ('37', '37'),
 ('23', '23'),
 ('39', '39'),
 ('30', '30'),
 ('23', '23'),
 ('23', '23'),
 ('32', '32'),
 ('35', '35'),
 ('21', '21'),
 ('24', '24'),
 ('19', '19'),
 ('32', '32'),
 ('15', '15'),
 ('27', '27'),
 ('34', '34'),
 ('44', '44'),
 ('36', '36'),
 ('38', '38'),
 ('33', '33'),
 ('30', '30'),
 ('21', '21'),
 ('16', '16'),
 ('30', '3

In [25]:
1-np.abs((np.array(exp,dtype='int') - np.array(pred,dtype='int')).mean())

1.0